#### Workbook for figuring how to check if functions are present in library when accessing workflows

In [9]:
import importlib.util 

package= "skimage.measure.regionprops_table"
is_present = importlib.util.find_spec(package) #find_spec will look for the package
print(is_present)


ModuleSpec(name='skimage', loader=<_frozen_importlib_external.SourceFileLoader object at 0x000001F72230C550>, origin='c:\\Users\\Pradeep\\.conda\\envs\\aicsimageio_test\\lib\\site-packages\\skimage\\__init__.py', submodule_search_locations=['c:\\Users\\Pradeep\\.conda\\envs\\aicsimageio_test\\lib\\site-packages\\skimage'])


In [1]:
from napari_workflows import Workflow
import pyclesperanto_prototype as cle
image_seg_workflow = Workflow()
from skimage.io import imread

img = imread("../sample_data/RBC_lattice.tif")

voxel_size_x_in_microns = 0.145
voxel_size_y_in_microns = 0.145
voxel_size_z_in_microns = 0.3
deskewing_angle_in_degrees = 30

input_arg = "input"
image_seg_workflow.set("deskewing", cle.deskew_y, input_arg, angle_in_degrees = deskewing_angle_in_degrees,
                    voxel_size_x = voxel_size_x_in_microns, voxel_size_y= voxel_size_y_in_microns,
                    voxel_size_z = voxel_size_z_in_microns)

image_seg_workflow.set("median", cle.median_sphere,"deskewing",radius_x = 2, radius_y = 2, radius_z = 2)

image_seg_workflow.set("binarisation", cle.threshold,"median",constant =1000)

image_seg_workflow.set("labeling", cle.connected_components_labeling_box,"binarisation")

print(str(image_seg_workflow))

Workflow:
deskewing <- (<function deskew_y at 0x0000024BBA0070D0>, 'input', None, 30, 0.145, 0.145, 0.3, 1)
median <- (<function median_sphere at 0x0000024BB8C70D30>, 'deskewing', None, 2, 2, 2)
binarisation <- (<function greater_constant at 0x0000024BB8C641F0>, 'median', None, 1000)
labeling <- (<function connected_components_labeling_box at 0x0000024BB8D1D1F0>, 'binarisation')



In [ ]:
#Iterate through tasks, get function and check if available.  
#If not in environment, print error, this needs to be tested before running pipeline

In [26]:
scale = (1,2,3)
import numpy as np
ase = list(scale[1:3])
print(ase)
ase.insert(0,1)
print(ase)


[2, 3]
[1, 2, 3]


In [23]:
ase

[1, 2, 3]